In [32]:
import pandas as pd
import numpy as np
import os

import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree, naive_bayes
import numpy as np
import seaborn as sns
import re
from scipy.stats import chi2_contingency
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder, StandardScaler
import graphviz
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, balanced_accuracy_score
import optuna
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import spacy
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

C:\Users\Bryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bryan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Bryan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bryan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# acllmdb Dataset

## Convert to CSV

In [19]:
def unison_shuffled_copies(a, b, c, random_state=42):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    assert len(a) == len(b)
    assert len(a) == len(c)
    p = np.random.RandomState(seed=random_state).permutation(len(a))
    return a[p], b[p], c[p]

def read_contents_update_array(Dirname, filename, textArray, decimalLabelArray, integerLabelArray):
    originalLabel = filename.split("_")[1].split(".")[0]
    normalizedLabel = (int(originalLabel) - 1) / 9
    decimalLabelArray.append(normalizedLabel)
    integerLabelArray.append(round(normalizedLabel))
    
    with open(Dirname + filename, 'r', encoding='latin-1') as f:
        textArray.append(f.read())

    return textArray, decimalLabelArray, integerLabelArray

In [27]:
# group the supervised into train and test csv
# rating is 1 - 10
# for better scale, we normalize the data to between 0 - 1
textArray = []
decimalLabelArray = []
integerLabelArray = []
motherDir = "./aclImdb/"
for filename in os.listdir(motherDir + "train/pos"):
    textArray, decimalLabelArray, integerLabelArray = read_contents_update_array(motherDir + "train/pos/", filename, textArray, decimalLabelArray, integerLabelArray)
print("Reach here")
for filename in os.listdir(motherDir + "train/neg"):
    textArray, decimalLabelArray, integerLabelArray = read_contents_update_array(motherDir + "train/neg/", filename, textArray, decimalLabelArray, integerLabelArray)

textArray, decimalLabelArray, integerLabelArray = unison_shuffled_copies(textArray, decimalLabelArray, integerLabelArray)

Reach here


In [28]:
train_df = pd.DataFrame({'text': textArray, 'decimalLabel': decimalLabelArray, 'integerLabel': integerLabelArray})
train_df.to_csv("aclimdb/train.csv", index=False)

In [24]:
# group the supervised into train and test csv
# rating is 1 - 10
# for better scale, we normalize the data to between 0 - 1
textArray = []
decimalLabelArray = []
integerLabelArray = []
motherDir = "./aclImdb/"
for filename in os.listdir(motherDir + "test/pos"):
    textArray, decimalLabelArray, integerLabelArray = read_contents_update_array(motherDir + "test/pos/", filename, textArray, decimalLabelArray, integerLabelArray)
print("Reach here")
for filename in os.listdir(motherDir + "test/neg"):
    textArray, decimalLabelArray, integerLabelArray = read_contents_update_array(motherDir + "test/neg/", filename, textArray, decimalLabelArray, integerLabelArray)

textArray, decimalLabelArray, integerLabelArray = unison_shuffled_copies(textArray, decimalLabelArray, integerLabelArray)

Reach here


In [26]:
test_df = pd.DataFrame({'text': textArray, 'decimalLabel': decimalLabelArray, 'integerLabel': integerLabelArray})
test_df.to_csv("aclimdb/test.csv", index=False)

## Read CSV, create heuristic characteristic for analysis

In [31]:
train_acl = pd.read_csv("aclimdb/train.csv")
test_acl = pd.read_csv("aclimdb/test.csv")

In [33]:
#Try to extract several variables from text
#simple features
def splitSentenceInPredefinedSeparator(x):
    separators = [',', ' ', '.']
    for separator in separators:
        x = x.replace(separator, ' ')
    return str(x).split()

#Count number of words
def numberOfWords(x):
    #return number of words that has been splitted including with ',', '.', ' '
    return len(splitSentenceInPredefinedSeparator(x))

def uniqueWords(x):
    #return number of unique words that has been splitted including with ',', '.', ' '
    return len(set(splitSentenceInPredefinedSeparator(x)))


train_acl['num-words'] = train_acl['text'].apply(lambda x: numberOfWords(x))
test_acl['num-words'] = test_acl['text'].apply(lambda x: numberOfWords(x))
#Count number of unique words
train_acl['num-unique-words'] = train_acl['text'].apply(lambda x: uniqueWords(x))
test_acl['num-unique-words'] = test_acl['text'].apply(lambda x: uniqueWords(x))
#Count number of punctuations
train_acl['num-punctuations'] = train_acl['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
test_acl['num-punctuations'] = test_acl['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
#UniqueWordRatio
train_acl['unique-word-ratio'] = train_acl['num-unique-words'] / train_acl['num-words']
test_acl['unique-word-ratio'] = test_acl['num-unique-words'] / test_acl['num-words']
#PunctuationRatio
train_acl['punctuation-ratio'] = train_acl['num-punctuations'] / train_acl['num-words']
test_acl['punctuation-ratio'] = test_acl['num-punctuations'] / test_acl['num-words']
#Sentence Length
train_acl['sentence-length'] = train_acl['text'].apply(lambda x: len(x.split('.')))
test_acl['sentence-length'] = test_acl['text'].apply(lambda x: len(x.split('.')))
#Number of Words beginning with capital letters (Other than first word in a sentence after period)
train_acl['num-words-beginning-capital'] = train_acl['text'].apply(lambda x: len([w for w in x.split() if w[0].isupper() and w[0] not in ['.', '"', "'"]]))
test_acl['num-words-beginning-capital'] = test_acl['text'].apply(lambda x: len([w for w in x.split() if w[0].isupper() and w[0] not in ['.', '"', "'"]]))
#Count Number of Nouns, and Nouns Ratio
print("beginning noun extraction")
train_acl['num-nouns'] = train_acl['text'].apply(lambda x: len([w for w in word_tokenize(x) if pos_tag([w])[0][1] in ['NN', 'NNP']]))
test_acl['num-nouns'] = test_acl['text'].apply(lambda x: len([w for w in word_tokenize(x) if pos_tag([w])[0][1] in ['NN', 'NNP']]))
train_acl['nouns-ratio'] = train_acl['num-nouns'] / train_acl['num-words']
test_acl['nouns-ratio'] = test_acl['num-nouns'] / test_acl['num-words']
#Count Number of Verbs, and Verbs Ratio
print("beginning verb extraction")
train_acl['num-verbs'] = train_acl['text'].apply(lambda x: len([w for w in word_tokenize(x) if pos_tag([w])[0][1] in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']]))
test_acl['num-verbs'] = test_acl['text'].apply(lambda x: len([w for w in word_tokenize(x) if pos_tag([w])[0][1] in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']]))
train_acl['verbs-ratio'] = train_acl['num-verbs'] / train_acl['num-words']
test_acl['verbs-ratio'] = test_acl['num-verbs'] / test_acl['num-words']
#Count Number of Adjectives, and Adjectives Ratio
print("beginning adjective extraction")
train_acl['num-adjectives'] = train_acl['text'].apply(lambda x: len([w for w in word_tokenize(x) if pos_tag([w])[0][1] in ['JJ', 'JJR', 'JJS']]))
test_acl['num-adjectives'] = test_acl['text'].apply(lambda x: len([w for w in word_tokenize(x) if pos_tag([w])[0][1] in ['JJ', 'JJR', 'JJS']]))
train_acl['adjectives-ratio'] = train_acl['num-adjectives'] / train_acl['num-words']
test_acl['adjectives-ratio'] = test_acl['num-adjectives'] / test_acl['num-words']
#Average Word Length
train_acl['average-word-length'] = train_acl['text'].apply(lambda x: np.mean([len(w) for w in splitSentenceInPredefinedSeparator(x)]))
test_acl['average-word-length'] = test_acl['text'].apply(lambda x: np.mean([len(w) for w in splitSentenceInPredefinedSeparator(x)]))
#stopwords count
train_acl['stopwords-count'] = train_acl['text'].apply(lambda x: len([w for w in splitSentenceInPredefinedSeparator(x) if w.lower() in nltk.corpus.stopwords.words('english')]))
test_acl['stopwords-count'] = test_acl['text'].apply(lambda x: len([w for w in splitSentenceInPredefinedSeparator(x) if w.lower() in nltk.corpus.stopwords.words('english')]))
train_acl['Stopwords-ratio'] = train_acl['stopwords-count'] / test_acl['num-words']
test_acl['Stopwords-ratio'] = test_acl['stopwords-count'] / test_acl['num-words']

beginning noun extraction


## Feature Engineering: TF-IDF

In [ ]:
train_acl.to_csv("aclimdb/train_heuristic.csv", index=False)
test_acl.to_csv("aclimdb/test_heuristic.csv", index=False)

# Sentiment Labelled Sentences Dataset